In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm
from scipy.signal import savgol_filter
import pandas as pd

from scipy.stats import linregress
from scipy.interpolate import RegularGridInterpolator
from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator,  FormatStrFormatter
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io
import math

plt.rcParams['font.size'] = 14

In [ ]:
def createSimDic(sim_labels, test):
    sim = {}
    for key in sim_labels.keys():
        sim[key] = {}
        for dtw in sim_labels[key]:
            sim[key][dtw] = ou.Simulation(f"/home/exxxx5/Tese/Decks/weibelTestsFinal/{test}/{key}/dtw{dtw}/{key}.in")
    return sim


def round_size(n):
    """Round n to 1 significant digit."""
    if n == 0:
        return 0
    return round(n, -int(math.floor(math.log10(n))))

def mean_rel_err_with_ar1(e):
    e = np.asarray(e)              # 1) Ensure e is a NumPy array (vector ops, speed, safety).
    N = e.size                     # 2) Number of time samples.

    if N < 2:                      # 3) If we have <2 points, we can't estimate variability.
        return float(e.mean()), np.nan

    em = e.mean()                  # 4) Sample mean: this is the point you will plot.
    x = e - em                     # 5) Mean-center the series (needed for variance & autocorr).
    s2 = x.var(ddof=1)             # 6) Sample variance of e_t (unbiased: ddof=1).

    # 7) Lag-1 autocorrelation ρ(1): similarity between consecutive samples.
    #    num = Σ (x_t * x_{t-1}); den = Σ (x_t^2). ρ(1) = num/den.
    num = np.dot(x[1:], x[:-1])
    den = np.dot(x, x)
    if (den == 0 or N < 3):
        rho1 = 0.0                 #    If constant or too short, assume no autocorrelation.
    else:
        rho1 = np.clip(num / den, -0.99, 0.99)  #    Clip for numerical stability.

    # 8) Effective sample size Neff for AR(1)-like correlation:
    #    With positive ρ(1), neighboring points carry redundant info, so Neff < N.
    Neff = N * (1 - rho1) / (1 + rho1)
    Neff = float(np.clip(Neff, 1.0, N))   #    Keep in [1, N] to avoid pathologies.

    # 9) Standard error of the mean using Neff (not naive N):
    #    SE( ē ) ≈ sqrt( s^2 / Neff ).
    se = np.sqrt(s2 / Neff)

    return float(em), float(se)

def find_nans_2d(a: np.ndarray):
    """
    Return locations and summary of NaNs in a 2D array.

    Returns dict with:
      - mask: boolean array, True where NaN
      - coords: (k,2) int array of [row, col] indices
      - rows: unique row indices containing NaN
      - cols: unique col indices containing NaN
      - count: total number of NaNs
    """
    a = np.asarray(a)
    if a.ndim != 2:
        raise ValueError("Input must be 2D")

    mask = np.isnan(a)
    count = int(mask.sum())
    if count:
        coords = np.argwhere(mask)
        rows = np.unique(coords[:, 0])
        cols = np.unique(coords[:, 1])
    else:
        coords = np.empty((0, 2), dtype=int)
        rows = np.array([], dtype=int)
        cols = np.array([], dtype=int)

    return dict(mask=mask, coords=coords, rows=rows, cols=cols, count=count)

In [ ]:
def check_out_bounds(track):
    """
    Return rows of `track` that are outside the grid and a boolean flag
    indicating whether at least one particle is out of bounds.
    """
    mask = ((track['x1'] < track.grid[0, 0]) | (track['x1'] > track.grid[0, 1]) |
            (track['x2'] < track.grid[1, 0]) | (track['x2'] > track.grid[1, 1]) |
            (track['x3'] < track.grid[2, 0]) | (track['x3'] > track.grid[2, 1]))

    has_oob = bool(mask.any())
    return has_oob

In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, tmin = 0, tmax = 100000000000, markersize=1.8, color = None, linewidth=0.8, linestyle='-', unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label2 = ("$t = {:.3f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle=linestyle, markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.set_ylabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x2"]))
    ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]))
    ax.legend()

    return fig, ax

In [ ]:



def xRelErr(sim, baseline, particles=None,
            dtw_list=("100", "50", "10", "1", "0_1"),
            index_min=1, box_length=None):
    """
    Plot full-position relative distance error with periodic (cubic) box.
    Distances use the minimal-image convention and are normalized by L.
    """

    import numpy as np
    import matplotlib.pyplot as plt

    # ---- helper to normalize 'particles' ----
    def _normalize_particles(sel, Np):
        if sel is None:
            return np.arange(Np, dtype=int)
        if isinstance(sel, slice):
            return np.arange(Np, dtype=int)[sel]
        arr = np.asarray(sel)
        if arr.dtype == bool:
            if arr.size != Np:
                raise ValueError(f"Boolean mask size {arr.size} != Np {Np}")
            return np.flatnonzero(arr)
        return np.atleast_1d(arr).astype(int)

    # ---- minimal-image delta under periodic BCs ----
    def _min_image(d, L):
        # maps differences to [-L/2, L/2] via nearest image
        return d - L * np.round(d / L)

    fig, ax = plt.subplots()

    # ---- read baseline once ----
    bx1_all = baseline["test_electrons"]["tracks"]["x1"]  # (Np, Tb)
    bx2_all = baseline["test_electrons"]["tracks"]["x2"]
    bx3_all = baseline["test_electrons"]["tracks"]["x3"]
    Np, Tb = bx1_all.shape

    part_idx = _normalize_particles(particles, Np)

    # try to infer box length L if not provided
    if box_length is None:
        L = baseline["test_electrons"]["tracks"].grid[0, 1] - baseline["test_electrons"]["tracks"].grid[0, 0]
    else:
        L = float(box_length)

    # trim baseline consistently
    bN = round_size(Tb) + 1
    bx1_all = bx1_all[:, :bN][part_idx]
    bx2_all = bx2_all[:, :bN][part_idx]
    bx3_all = bx3_all[:, :bN][part_idx]

    # init NaN trackers (counts among selected particles)
    nan_particle_count = {p: {dtw: 0 for dtw in sim[p]} for p in sim}
    nan_particle_list  = {p: {dtw: [] for dtw in sim[p]} for p in sim}

    for pusher in sim.keys():
        X, Y, YERR, YMax = [], [], [], []

        for dtw in dtw_list:
            step = max(1, int(float(dtw.replace("_", "."))))

            # baseline downsampled to match sim stride (vectorized)
            bx1 = bx1_all[:, step::step]
            bx2 = bx2_all[:, step::step]
            bx3 = bx3_all[:, step::step]

            # sim for all selected particles
            sx1_all = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][part_idx]
            sx2_all = sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][part_idx]
            sx3_all = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][part_idx]

            # trim sim and apply index_min
            Ts = sx1_all.shape[1]
            sN = round_size(Ts) + 1
            sx1 = sx1_all[:, index_min:sN]
            sx2 = sx2_all[:, index_min:sN]
            sx3 = sx3_all[:, index_min:sN]

            # ensure equal time length
            Ltime = min(bx1.shape[1], sx1.shape[1])
            if Ltime <= 0:
                continue
            bx1, bx2, bx3 = bx1[:, :Ltime], bx2[:, :Ltime], bx3[:, :Ltime]
            sx1, sx2, sx3 = sx1[:, :Ltime], sx2[:, :Ltime], sx3[:, :Ltime]

            # skip particles with ANY NaN in sim positions
            bad = (np.isnan(sx1).any(axis=1) |
                   np.isnan(sx2).any(axis=1) |
                   np.isnan(sx3).any(axis=1))
            nan_particle_count[pusher][dtw] = int(bad.sum())
            nan_particle_list[pusher][dtw]  = np.asarray(part_idx)[bad].tolist()

            good = ~bad
            if not np.any(good):
                continue

            # select good particles
            sx1g, sx2g, sx3g = sx1[good], sx2[good], sx3[good]
            bx1g, bx2g, bx3g = bx1[good], bx2[good], bx3[good]

            # ---- periodic minimal-image distance ----
            dx1 = _min_image(sx1g - bx1g, L)
            dx2 = _min_image(sx2g - bx2g, L)
            dx3 = _min_image(sx3g - bx3g, L)
            dist = np.sqrt(dx1**2 + dx2**2 + dx3**2)  # (Ng, T)

            # normalize by box length (fraction of box)
            rel = dist / L

            # one scalar per particle (mean over time), then stats across particles
            m = rel.mean(axis=1)   # (Ng,)
            X.append(float(dtw.replace("_", ".")))
            Y.append(m.mean())
            YERR.append(m.std(ddof=1))
            YMax.append(m.max())

        linestyle = (0, (4, 4)) if pusher == "GcaCorr" else '-'

        eb = ax.errorbar(
            X, Y, yerr=YERR,
            label=fr"{pusher}",
            fmt='o',
            linestyle=linestyle,
            markersize=3.5,
            linewidth=1.2,
            capsize=3,
            elinewidth=1.0,
        )

        ax.plot(
            np.asarray(X, dtype=float),
            np.asarray(YMax, dtype=float),
            marker='x',
            linestyle='none',
            markersize=5,
            markeredgewidth=1.0,
            zorder=3,
            color=eb.lines[0].get_color(),
        )

    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
    ax.set_ylabel(r"$x$ relative error")
    ax.legend()
    ax.grid()

    return fig, ax, nan_particle_count, nan_particle_list



## Bx100, Ex0.1

In [ ]:
dtw = "0_01"
# test = "NoE_FinalV"
test = "OriginalParams_FinalV"
sim_labels = {
    'Boris': ["100", "50", "10", "1", "0_1"],
    'Gca': ["100", "50", "10", "1", "0_1"],
    'GcaCorr': ["100", "50", "10", "1", "0_1"],
}

# sametimeIter ={
#     "1000" : 10,
#     "100" : 100,
#     "10" : 1000,
#     "1" : 10000,
#     "0_1" : 10000,
# }

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/weibelTestsFinal/{test}/Boris/dtw{dtw}/Boris.in")
baseline["test_electrons"]["tracks"].load_all()

print("Check out of bounds particles:")
for pusher in sim.keys():
    for dtw in ["100", "50", "10"]:
        if(check_out_bounds(sim[pusher][dtw]["test_electrons"]["tracks"])):
            print(f"  {pusher} dtw{dtw}: OUT OF BOUNDS PARTICLES DETECTED!")

In [ ]:
# particle = 4


# fig, ax = None, None
# tmin = 5
# tmax=628.31859
# dtw = "100"
# for pusher in ["Gca"]:
#     # for dtw in sim[pusher].keys():
#     fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", particle=particle, tmin=tmin, tmax=tmax, fig=fig, ax=ax)

# fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, color="black", markersize=0, linewidth=0.6, tmin=tmin, tmax=tmax, fig=fig, ax=ax)
# # ax.view_init(elev=0, azim=80)
# # ax.tick_params(axis='x', pad=15)
# # ax.tick_params(axis='y', pad=15)
# ax.tick_params(axis='z', pad=10)

# # ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# ax.zaxis.set_major_locator(plt.MaxNLocator(4))

# ax.xaxis.labelpad = 10
# ax.yaxis.labelpad = 10
# # ax.zaxis.labelpad = 1



# ax.legend()
# plt.show()

In [ ]:
fig, ax, nan_counts, nan_lists = xRelErr(sim, baseline, particles=np.arange(1000), dtw_list=("100","50","10","1"))
# ax.set_ylim(1e-5, 1e1)
plt.show()

print(nan_counts)

## Bx100, Ex0

In [ ]:
dtw = "0_01"
test = "NoE_FinalV"
# test = "OriginalParams_FinalV"
sim_labels = {
    'Boris': ["100", "50", "10", "1", "0_1"],
    'Gca': ["100", "50", "10", "1", "0_1"],
    'GcaCorr': ["100", "50", "10", "1", "0_1"],
}

# sametimeIter ={
#     "1000" : 10,
#     "100" : 100,
#     "10" : 1000,
#     "1" : 10000,
#     "0_1" : 10000,
# }

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/weibelTestsFinal/{test}/Boris/dtw{dtw}/Boris.in")
baseline["test_electrons"]["tracks"].load_all()

print("Check out of bounds particles:")
for pusher in sim.keys():
    for dtw in ["100", "50"]:
        if(check_out_bounds(sim[pusher][dtw]["test_electrons"]["tracks"])):
            print(f"  {pusher} dtw{dtw}: OUT OF BOUNDS PARTICLES DETECTED!")

In [ ]:
nans = find_nans_2d(sim["Gca"]["100"]["test_electrons"]["tracks"]["x1"][:,:])

print(nans["rows"].shape)

In [ ]:
fig, ax, nan_counts, nan_lists = xRelErr(sim, baseline, particles=np.arange(1000), dtw_list=("100","50","10","1"))
# ax.set_ylim(1e-5, 1e1)
plt.show()

print(nan_counts)

## Bx100, Ex0.1, Lx10

In [ ]:
dtw = "0_01"
test = "L400_FinalV"
# test = "OriginalParams_FinalV"
sim_labels = {
    'Boris': ["100", "50", "10", "1", "0_1"],
    'Gca': ["100", "50", "10", "1", "0_1"],
    'GcaCorr': ["100", "50", "10", "1", "0_1"],
}

# sametimeIter ={
#     "1000" : 10,
#     "100" : 100,
#     "10" : 1000,
#     "1" : 10000,
#     "0_1" : 10000,
# }

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/weibelTestsFinal/{test}/Boris/dtw{dtw}/Boris.in")
baseline["test_electrons"]["tracks"].load_all()

print("Check out of bounds particles:")
for pusher in sim.keys():
    for dtw in ["100", "50"]:
        if(check_out_bounds(sim[pusher][dtw]["test_electrons"]["tracks"])):
            print(f"  {pusher} dtw{dtw}: OUT OF BOUNDS PARTICLES DETECTED!")

In [ ]:
fig, ax, nan_counts, nan_lists = xRelErr(sim, baseline, particles=np.arange(1000), dtw_list=("100","50","10","1"))
# ax.set_ylim(1e-5, 1e1)
plt.show()

print(nan_counts)

## Bx100, Ex0.1, Lx10

In [ ]:
dtw = "0_01"
# test = "NoE_FinalV"
test = "NoEL400_FinalV"
sim_labels = {
    'Boris': ["100", "50", "10", "1", "0_1"],
    'Gca': ["100", "50", "10", "1", "0_1"],
    'GcaCorr': ["100", "50", "10", "1", "0_1"],
}

# sametimeIter ={
#     "1000" : 10,
#     "100" : 100,
#     "10" : 1000,
#     "1" : 10000,
#     "0_1" : 10000,
# }

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/weibelTestsFinal/{test}/Boris/dtw{dtw}/Boris.in")
baseline["test_electrons"]["tracks"].load_all()

print("Check out of bounds particles:")
for pusher in sim.keys():
    for dtw in ["100", "50"]:
        if(check_out_bounds(sim[pusher][dtw]["test_electrons"]["tracks"])):
            print(f"  {pusher} dtw{dtw}: OUT OF BOUNDS PARTICLES DETECTED!")



In [ ]:
fig, ax, nan_counts, nan_lists = xRelErr(sim, baseline, particles=np.arange(1000), dtw_list=("100","50","10","1"))
# ax.set_ylim(1e-5, 1e1)
plt.show()

print(nan_counts)